## Import Statements

In [1]:
import numpy as np
import csv
import pandas as pd
import pylab 
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
plt.rcParams["font.family"] = "Times New Roman"

# NOTE: you also need to have the beam model set up on your computer so you can directly import it 

In [2]:
# csv file columns
col_list = ['psrName', 'RA_EF','Dec _EF']

In [3]:
# reading in the csv file
df = pd.read_csv("/Users/curtina/Documents/McGill/CHIME/Outriggers/PSR_Sky_Dist/VLBA_PSR_RA_DEC.csv",usecols=col_list)

# Sky Distribution

In [9]:
# plotting psr in ra, dec space
# only plotting PSR which have been localized, plus those by Adam Deller plus the handful we added to fill in space
plt.rcParams.update({'font.size': 30})
fig, ax = plt.subplots()
plt.xlabel("RA (deg)")
plt.ylabel("DEC (deg)")
plt.xlim(0,360)
fig.set_size_inches(30, 10)
# iterating through each pulsar in CSV
for index, row in df.iterrows():
    ra_i = row['RA_EF']
    dec_i = row['Dec _EF']
    coord = SkyCoord(ra_i,dec_i, unit=(u.hourangle, u.deg))
    ra = coord.ra.degree
    dec = coord.dec.degree
    # calculating time within FoV in both upper and lower transits
    before_min_up, after_min_up = get_transit_duration(dec, 'U')
    before_min_low, after_min_low = get_transit_duration(dec, 'L')
    before_deg_up = ra - before_min_up/1440*360
    after_deg_up =  ra + after_min_up/1440*360
    if before_min_low > 0:
        if ra > 180:
            ra_low = ra - 180
            before_deg_low = ra_low - before_min_low/1440*360
            after_deg_low = ra_low + after_min_low/1440*360
        else:
            ra_low = ra + 180
            before_deg_low = ra_low - before_min_low/1440*360
            after_deg_low = ra_low + after_min_low/1440*360
    else:
        before_deg_low = 0
        after_deg_low = 0
    
    # not plotting PSR which we haven't localized, but plotting those which
    # Adam Deller found
    if (row['psrName'] in psr_no_vlba and row['psrName'] not in deller_psr):
        continue
    else:
        ax.axvspan(before_deg_up, after_deg_up, alpha=0.4, color='grey')
        ax.axvspan(before_deg_low, after_deg_low, alpha=0.4, color='grey')
        plt.plot(ra,dec, "ok", markersize = 14)
        if before_min_low > 0:
            plt.plot(ra_low,dec, "ok", markersize = 14, mfc='none')

    
# four pulsars that we manually added to VLBA observations
ras = [40.646, 40.75, 218.498, 349.5]
decs = [62.9, 60.45, 72.9, 21.63]

for idx in range(len(ras)):
    ra = ras[idx]
    dec = decs[idx]
    # calculating time within FoV in both upper and lower transits
    before_min_up, after_min_up = get_transit_duration(dec, 'U')
    before_min_low, after_min_low = get_transit_duration(dec, 'L')
    before_deg_up = ra - before_min_up/1440*360
    after_deg_up =  ra + after_min_up/1440*360
    if before_min_low > 0:
        if ra > 180:
            ra_low = ra - 180
            before_deg_low = ra_low - before_min_low/1440*360
            after_deg_low = ra_low + after_min_low/1440*360
        else:
            ra_low = ra + 180
            before_deg_low = ra_low - before_min_low/1440*360
            after_deg_low = ra_low + after_min_low/1440*360
    else:
        before_deg_low = 0
        after_deg_low = 0
    
    ax.axvspan(before_deg_up, after_deg_up, alpha=0.4, color='grey')
    ax.axvspan(before_deg_low, after_deg_low, alpha=0.4, color='grey')
    plt.plot(ra,dec, "ok", markersize = 14)
    if before_min_low > 0:
        plt.plot(ra_low,dec, "ok", markersize = 14, mfc='none')


legend_elements = [Line2D([0], [0], marker='o', color='w', label='Upper transit',
                          markerfacecolor='k', markersize=14),
                   Line2D([0], [0], marker='o', color='w', label='Lower transit',
                          markerfacecolor='none', markersize=14, markeredgecolor = 'k'),
                   Patch(facecolor='red', alpha = 0.4, edgecolor='r',
                         label='Sky Coverage')]

#plt.legend(handles=legend_elements, loc=(1.01,.5))
plt.savefig('psr_sky_dist.png', dpi=200, bbox_inches='tight')

## Some defined functions and arrays that are needed

In [5]:
import datetime as dt
import ephem
import pytz
import astropy.units as u
import numpy as np
def get_transit_duration(dec, transit = 'U'):
    """Return duration source is observable for before and after transit at meridian.
    
    Parameters
    ----------
    decjd : float
        Declination, in degrees.
    transit : str
        'U' for upper and 'L' is lower
    Returns
    -------
    before_t_mins : float
        Time source is observable before transit at meridian
    after_t_mins : float
        Time source is observable after transit at meridian
    """
    ew_beam_width = 0.32044866 #deg.
    before_ext = 1.5 * ew_beam_width
    after_ext = 2.5 * ew_beam_width
    # 1440 minutes per day
    # 1.5 beam widths before at meridian and 2.5 after since transit is in first column
    before_t_mins = 1440. * (before_ext / np.cos(np.deg2rad(dec))) / 360.
    after_t_mins = 1440. * (after_ext / np.cos(np.deg2rad(dec))) / 360.
    
    #Accounting for singularity at dec = 90.
    before_max = (1.5/4.) * 0.5 * 1440
    after_max = (2.5/4.) * 0.5 * 1440
    if before_t_mins > before_max:
        before_t_mins = before_max
    if after_t_mins > after_max:
        after_t_mins = after_max
    
    if transit == 'U':
        return before_t_mins, after_t_mins
    elif transit == 'L':
        if dec < 70:
            return 0, 0
        else:
            return after_t_mins, before_t_mins #source enters the beam from the west

# UNLOCALIZED, DELLER, MSP, AND BINARY PSR

In [7]:
# pulsars which we stopped observing with the VLBA (either couldn't localize or Deller PSR)
psr_no_vlba = [
"B0037+56",
"B0154+61",
"B1508+55",
"B2303+30",
"B2351+61",
"J0407+1607",
"J0125+62",
"B0823+26",
"B0329+54",
"B0353+52",
"J1022+1001",
"B1133+16",
"B1322+83",
"B2148+52",
"J2241+6941",
"B1957+20",
"B2053+21",
"B2110+27",
"B2210+29",
"J1710+4923",
"B1753+52",
"B1839+56",
"J0032+6946",
"J1810+1744",
"J1706+59",
]

# pulsars that were localized by Adam Deller, so we are including all of them (even if)
# we couldn't localize them
deller_psr = ['B0037+56',
'B0105+65',
'B0154+61',
'B0320+39',
'B0329+54',
'B0353+52',
'B0823+26',
'J1022+1001',
'B1133+16',
'B1322+83',
'B1530+27',
'B1753+52',
'B1839+56'    
]

In [6]:
get_transit_duration(61.89, transit = 'U')


(4.080711758640706, 6.801186264401177)

In [19]:
dec = 65.16
before_ext = 17
after_ext = 17
# 1440 minutes per day
# 1.5 beam widths before at meridian and 2.5 after since transit is in first column
before_t_mins = 1440. * (before_ext / np.cos(np.deg2rad(dec))) / 360.
after_t_mins = 1440. * (after_ext / np.cos(np.deg2rad(dec))) / 360.

print((before_t_mins + after_t_mins)/60)

5.395724047175135
